In [40]:
import logging
from hermes.app import HermesApp
from hermes.ui_agent import LLMReasoningCapture
# Initialize the reasoning capture handler
reasoning_capture = LLMReasoningCapture()

# Attach to relevant loggers
for logger_name in ['root', 'hermes']:
    logging.getLogger(logger_name).addHandler(reasoning_capture)
    
hermes_app = HermesApp()

In [41]:
hermes_app.get_csv_files()
hermes_app.load_data(None, None, selected_file="data/shipments.csv")

(       id    route warehouse  delivery_time  delay_minutes      delay_reason  \
 0       1  Route C       WH2           4.88             51     Customs Delay   
 1       2  Route A       WH4           4.77             45       Driver Rest   
 2       3  Route C       WH4           5.31            113           Traffic   
 3       4  Route B       WH4           5.83             32   Minor Breakdown   
 4       5  Route C       WH4           6.13            120           Traffic   
 ..    ...      ...       ...            ...            ...               ...   
 995   996  Route B       WH2           5.60            118           Weather   
 996   997  Route E       WH4           5.00             83       Driver Rest   
 997   998  Route B       WH4           5.45             52       Driver Rest   
 998   999  Route E       WH4           5.61            117           Traffic   
 999  1000  Route B       WH3           5.52            105  Mechanical Issue   
 
           date  on_time  

In [3]:
# response = hermes_app._handle_general_chat("Show me the distribution of shipment delays.")

In [4]:
# response.metadata["raw"]

In [42]:
response = hermes_app._handle_stats_chat("Show me the distribution of shipment delays.")

In [6]:
response.raw_result.last_code_executed

'import pandas as pd\nsql_query = """\nSELECT \n    delay_reason, \n    COUNT(*) AS count\nFROM \n    table_6619b1d3d4c59fcb9f82845d59178bfd\nGROUP BY \n    delay_reason\nORDER BY \n    count DESC;\n"""\ndelay_distribution_df = execute_sql_query(sql_query)\nresult = {\'type\': \'dataframe\', \'value\': delay_distribution_df}'

In [43]:
reasoning_capture.categorized_steps['other']

[{'timestamp': '2025-11-09T23:39:02.861362',
  'elapsed_ms': 0,
  'level': 'WARNING',
  'logger': 'semantic',
  'message': "Failed to register semantic dataset: create() got an unexpected keyword argument 'name'",
  'step': 'other'},
 {'timestamp': '2025-11-09T23:39:02.861362',
  'elapsed_ms': 0,
  'level': 'WARNING',
  'logger': 'semantic',
  'message': "Failed to register semantic dataset: create() got an unexpected keyword argument 'name'",
  'step': 'other'},
 {'timestamp': '2025-11-09T23:39:02.866478',
  'elapsed_ms': 5,
  'level': 'INFO',
  'logger': 'app',
  'message': 'Loaded 1000 records from data/shipments.csv',
  'step': 'other'},
 {'timestamp': '2025-11-09T23:39:02.866478',
  'elapsed_ms': 5,
  'level': 'INFO',
  'logger': 'app',
  'message': 'Loaded 1000 records from data/shipments.csv',
  'step': 'other'},
 {'timestamp': '2025-11-09T23:39:08.175502',
  'elapsed_ms': 5314,
  'level': 'INFO',
  'logger': 'logger',
  'message': 'Running PandasAI with litellm LLM...',
  'step

In [45]:
reasoning_capture.categorized_steps

{'other': [{'timestamp': '2025-11-09T23:39:02.861362',
   'elapsed_ms': 0,
   'level': 'WARNING',
   'logger': 'semantic',
   'message': "Failed to register semantic dataset: create() got an unexpected keyword argument 'name'",
   'step': 'other'},
  {'timestamp': '2025-11-09T23:39:02.861362',
   'elapsed_ms': 0,
   'level': 'WARNING',
   'logger': 'semantic',
   'message': "Failed to register semantic dataset: create() got an unexpected keyword argument 'name'",
   'step': 'other'},
  {'timestamp': '2025-11-09T23:39:02.866478',
   'elapsed_ms': 5,
   'level': 'INFO',
   'logger': 'app',
   'message': 'Loaded 1000 records from data/shipments.csv',
   'step': 'other'},
  {'timestamp': '2025-11-09T23:39:02.866478',
   'elapsed_ms': 5,
   'level': 'INFO',
   'logger': 'app',
   'message': 'Loaded 1000 records from data/shipments.csv',
   'step': 'other'},
  {'timestamp': '2025-11-09T23:39:08.175502',
   'elapsed_ms': 5314,
   'level': 'INFO',
   'logger': 'logger',
   'message': 'Running 

In [38]:
# Show code generation step with formatted code
if 'code_generation' in reasoning_capture.categorized_steps:
    code_logs = reasoning_capture.categorized_steps['code_generation']
    display_prompt = ""
    duration = code_logs[-1]['elapsed_ms'] - code_logs[0]['elapsed_ms']
    # display_prompt += code_logs[0].get('message', '') + "\n" # 'Generating new code...'  command
    prompt = reasoning_capture.extract_code_from_message(code_logs[1]["message"], "using prompt:")
    prefix = code_logs[1]["message"]
    prefix = prefix.replace(prompt, "")
    display_prompt += f"{prefix}\n```html\n{prompt}```\n---\n" # the initial prompt
    
    code = reasoning_capture.extract_code_from_message(code_logs[2]["message"], "code generated:")
    prefix = code_logs[2]["message"]
    prefix = prefix.replace(code, "")
    display_prompt += f"{prefix}```python\n{code}```\n" # the generated code
    print(f"Code Generation Step (Duration: {duration}ms):\n{display_prompt}")

Code Generation Step (Duration: 2281ms):
Using Prompt: 
```html
<tables>

<table dialect="duckdb" table_name="table_6619b1d3d4c59fcb9f82845d59178bfd" columns="[{"name": "id", "type": "integer", "description": null, "expression": null, "alias": null}, {"name": "route", "type": "string", "description": null, "expression": null, "alias": null}, {"name": "warehouse", "type": "string", "description": null, "expression": null, "alias": null}, {"name": "delivery_time", "type": "float", "description": null, "expression": null, "alias": null}, {"name": "delay_minutes", "type": "integer", "description": null, "expression": null, "alias": null}, {"name": "delay_reason", "type": "string", "description": null, "expression": null, "alias": null}, {"name": "date", "type": "datetime", "description": null, "expression": null, "alias": null}, {"name": "on_time", "type": "integer", "description": null, "expression": null, "alias": null}]" dimensions="1000x8">
id,route,warehouse,delivery_time,delay_minute

In [12]:
reasoning_capture.categorized_steps['code_generation']

[{'timestamp': '2025-11-09T22:33:13.765689',
  'elapsed_ms': 287,
  'level': 'INFO',
  'logger': 'logger',
  'message': 'Generating new code...',
  'step': 'code_generation'},
 {'timestamp': '2025-11-09T22:33:13.793816',
  'elapsed_ms': 316,
  'level': 'INFO',
  'logger': 'logger',
  'message': 'Using Prompt: <tables>\n\n<table dialect="duckdb" table_name="table_6619b1d3d4c59fcb9f82845d59178bfd" columns="[{"name": "id", "type": "integer", "description": null, "expression": null, "alias": null}, {"name": "route", "type": "string", "description": null, "expression": null, "alias": null}, {"name": "warehouse", "type": "string", "description": null, "expression": null, "alias": null}, {"name": "delivery_time", "type": "float", "description": null, "expression": null, "alias": null}, {"name": "delay_minutes", "type": "integer", "description": null, "expression": null, "alias": null}, {"name": "delay_reason", "type": "string", "description": null, "expression": null, "alias": null}, {"name":

In [8]:
reasoning_capture.reasoning_logs

[{'timestamp': '2025-11-09T22:33:13.477696',
  'elapsed_ms': 0,
  'level': 'WARNING',
  'logger': 'semantic',
  'message': "Failed to register semantic dataset: create() got an unexpected keyword argument 'name'",
  'step': 'other'},
 {'timestamp': '2025-11-09T22:33:13.480142',
  'elapsed_ms': 2,
  'level': 'INFO',
  'logger': 'app',
  'message': 'Loaded 1000 records from data/shipments.csv',
  'step': 'other'},
 {'timestamp': '2025-11-09T22:33:13.507960',
  'elapsed_ms': 30,
  'level': 'INFO',
  'logger': 'app',
  'message': 'Handling statistics request (typed)',
  'step': 'query_understanding'},
 {'timestamp': '2025-11-09T22:33:13.509164',
  'elapsed_ms': 31,
  'level': 'INFO',
  'logger': 'logger',
  'message': 'Question: <TIME_CONTEXT>Current date: 2024-12-30</TIME_CONTEXT>\nShow me the distribution of shipment delays. /no_think',
  'step': 'query_understanding'},
 {'timestamp': '2025-11-09T22:33:13.728291',
  'elapsed_ms': 250,
  'level': 'INFO',
  'logger': 'logger',
  'message':

In [10]:
reasoning_capture._categorize_step("Executing code: print('Hello World')")

'code_execution'

In [46]:
from pandasai.core.response import DataFrameResponse, NumberResponse, ChartResponse, StringResponse

In [ ]:
DataFrameResponse.

TypeError: BaseResponse.to_json() missing 1 required positional argument: 'self'

In [23]:
response.__dict__

{'text': '🤖        delay_reason  count\n0              None    170\n1           Traffic    157\n2       Driver Rest    145\n3  Mechanical Issue    143\n4           Weather    137\n5   Minor Breakdown    133\n6     Customs Delay    115',
 'chart': None,
 'intent': 'general',
 'metadata': {'data_type': 'text',
  'raw': DataFrameResponse(type='dataframe', value=       delay_reason  count
  0              None    170
  1           Traffic    157
  2       Driver Rest    145
  3  Mechanical Issue    143
  4           Weather    137
  5   Minor Breakdown    133
  6     Customs Delay    115)},
 'timestamp': datetime.datetime(2025, 11, 6, 8, 53, 7, 296382),
 'success': True,
 'error': None,
 'data_type': 'text',
 'raw_result': DataFrameResponse(type='dataframe', value=       delay_reason  count
 0              None    170
 1           Traffic    157
 2       Driver Rest    145
 3  Mechanical Issue    143
 4           Weather    137
 5   Minor Breakdown    133
 6     Customs Delay    115)}

In [ ]:
from hermes.models import BaseResponse

response.chart.path if isinstance(response, BaseResponse) and response.chart else response.get("chart")
if isinstance(response, BaseResponse):
    if response.chart:
        return response.chart.path
else:
        return response.get("chart")

AttributeError: 'GeneralResponse' object has no attribute 'get'

In [29]:
response = hermes_app._handle_prediction_chat("Predict the delay rate for next week.")

TypeError: argument of type 'pydantic_core._pydantic_core.ValidationInfo' is not iterable

In [34]:
from hermes.config import CHARTS_DIR, DATA_DIR, QUESTIONS_FILE, SHIPMENTS_FILE, llm
from hermes.visualizer import HermesVisualizer
visualizer = HermesVisualizer(charts_dir="/data1/workspaces/phitha1/Hermes/exports/charts")

In [36]:
visualizer.get_latest_chart()

'/data1/workspaces/phitha1/Hermes/exports/charts/temp_chart_a22db8c9-d29c-4c67-a3e0-c664fbeacf8c.png'

In [7]:
from hermes.ui_agent import HermesAgentTools, extract_response_components
tools = HermesAgentTools(hermes_app)
# result = tools.analyze_query("Show delay reason counts for WH1.", "general")
result = tools.analyze_query("How many shipments from WH3 had delays?", "statistics")

In [8]:
result["response_model"].__dict__

{'text': '',
 'chart': None,
 'intent': 'statistics',
 'data_type': 'number',
 'raw_result': NumberResponse(type='number', value=203),
 'metadata': {'total_shipments': 1000,
  'delayed_shipments': 830,
  'on_time_shipments': 170,
  'on_time_rate': 0.17,
  'delay_rate': 0.83,
  'avg_delay_minutes': 63.59518072289157,
  'median_delay_minutes': 67.0,
  'avg_delivery_time': 5.49323,
  'date_range': '2024-01-01 to 2024-12-30'},
 'timestamp': datetime.datetime(2025, 11, 7, 15, 32, 34, 4285),
 'success': True,
 'error': None,
 'stats': StatsSummary(total_shipments=1000, delayed_shipments=830, on_time_shipments=170, on_time_rate=0.17, delay_rate=0.83, avg_delay_minutes=63.59518072289157, median_delay_minutes=67.0, avg_delivery_time=5.49323, date_range='2024-01-01 to 2024-12-30'),
 'data_preview': None}

In [11]:
extract_response_components(result["response_model"])

{'text': '',
 'dataframe': None,
 'number': 203.0,
 'chart_path': None,
 'metrics': None,
 'recommendations': None,
 'stats': {'total_shipments': 1000,
  'delayed_shipments': 830,
  'on_time_rate': 0.83,
  'avg_delay_minutes': 63.59518072289157},
 'data_type': 'number'}

In [12]:
if extract_response_components(result["response_model"])["data_type"] == "number" and extract_response_components(result["response_model"])["number"] is not None:
    print(extract_response_components(result["response_model"]))

{'text': '', 'dataframe': None, 'number': 203.0, 'chart_path': None, 'metrics': None, 'recommendations': None, 'stats': {'total_shipments': 1000, 'delayed_shipments': 830, 'on_time_rate': 0.83, 'avg_delay_minutes': 63.59518072289157}, 'data_type': 'number'}


In [2]:
from hermes.prompts import PROMPT_TEMPLATES
from hermes.llm import ask_llm

prompt = PROMPT_TEMPLATES['classification_intent'].format(query="List shipments in April with delay_minutes > 30.")
ask_llm(prompt)

'<think>\n\n</think>\n\n{"intent": "statistics", "confidence": 0.85}'

In [13]:
import pandas as pd

df = pd.read_csv("./data/shipments.csv")
df.drop_duplicates(inplace=True)
df.shape

(1000, 7)

In [17]:
import pandas as pd

questions = pd.read_csv("./data/questions.csv")

answers = []
for i, row in questions.iterrows():
    exec_locals = {"df": df, "pd": pd}
    exec(row["code_concise"], {}, exec_locals)
    answers.append(exec_locals.get("result", None))

<string>:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
<string>:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
<string>:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
<string>:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
<string>:3: FutureWarning: 'M' is deprecated and will be removed in a future version, 

In [23]:
pd.set_option("max_colwidth", None)
questions["answer"] = answers
questions.sample(5)

,id,question,code_concise,code_verbose,answer
134,405,Predict next week's delay rate using historical averages. (count only),df['date']=pd.to_datetime(df['date']); last_28=df[df['date']>df['date'].max()-pd.Timedelta(days=27)]; result = (last_28['delay_minutes']>0).mean(),df['date']=pd.to_datetime(df['date'])\nmax_date = df['date'].max()\nlast_28 = df[df['date'] > max_date - pd.Timedelta(days=27)]\nresult = (last_28['delay_minutes'] > 0).mean()\n,0.833333
9,10,What is the total delay time across all shipments?,result = df['delay_minutes'].sum(),# total delay minutes across dataset\nresult = df['delay_minutes'].sum()\n,52784
97,136,How many shipments occurred on Thursdays?,"result = {'shape': df.shape, 'columns': list(df.columns)}","result = {'shape': df.shape, 'columns': list(df.columns), 'sample': df.head(3).to_dict(orient='records')}\n","{'shape': (1000, 7), 'columns': ['id', 'route', 'warehouse', 'delivery_time', 'delay_minutes', 'delay_reason', 'date']}"
77,95,Which warehouse had the most shipments in May?,"result = {'shape': df.shape, 'columns': list(df.columns)}","result = {'shape': df.shape, 'columns': list(df.columns), 'sample': df.head(3).to_dict(orient='records')}\n","{'shape': (1000, 7), 'columns': ['id', 'route', 'warehouse', 'delivery_time', 'delay_minutes', 'delay_reason', 'date']}"
35,36,Which week had the most deliveries?,df['date']=pd.to_datetime(df['date']); result = df.groupby(df['date'].dt.isocalendar().week)['id'].count().idxmax(),df['date']=pd.to_datetime(df['date'])\n# week number with most deliveries\nresult = df.groupby(df['date'].dt.isocalendar().week)['id'].count().idxmax()\n,1
